## Imports + load .env + DB vars

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from pathlib import Path

# 1) Load .env
env_path = Path().resolve() / ".env"
print("Using .env from:", env_path)

load_dotenv(env_path, override=True)

# 2) Read DB credentials
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

print("DB_USER ->", DB_USER)
print("DB_HOST ->", DB_HOST)
print("DB_PORT ->", DB_PORT)
print("DB_NAME ->", DB_NAME)


Using .env from: /Users/vivekarya/Documents/GitHub/nem-usage-reports/.env
DB_USER -> vivekarya
DB_HOST -> localhost
DB_PORT -> 5432
DB_NAME -> postgres


## Step 2 : Create engine + test DB connection

In [2]:
engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

with engine.connect() as conn:
    result = conn.execute(text("SELECT 1;")).fetchall()
    print("✅ DB connection OK:", result)


✅ DB connection OK: [(1,)]


## Load daily_usage preview

In [3]:
df_daily = pd.read_sql("""
    SELECT *
    FROM daily_usage
    ORDER BY date, region_id
    LIMIT 10;
""", engine)

df_daily


,date,region_id,avg_5min_demand,daily_peak,daily_min,total_daily_energy_mwh
0,2025-01-01,NSW1,6774.125714,9434.59,4534.57,162014.506667
1,2025-01-01,QLD1,6039.234634,7808.79,4554.92,144438.361667
2,2025-01-01,SA1,862.286655,1518.36,10.55,20623.022500
3,2025-01-01,TAS1,999.820767,1104.47,920.09,23912.380000
4,2025-01-01,VIC1,3293.134390,4488.60,1400.13,78760.797500
5,2025-01-02,NSW1,6996.438160,8012.66,6219.28,167914.515833
6,2025-01-02,QLD1,6345.737014,7802.14,5272.72,152297.688333
7,2025-01-02,SA1,973.598368,1598.49,105.08,23366.360833
8,2025-01-02,TAS1,1004.650347,1131.23,843.71,24111.608333
9,2025-01-02,VIC1,3315.469653,4721.65,1646.40,79571.271667


## Load monthly_usage preview

In [4]:
df_monthly = pd.read_sql("""
    SELECT *
    FROM monthly_usage
    ORDER BY month, region_id
    LIMIT 10;
""", engine)

df_monthly


,month,region_id,avg_daily_demand,monthly_peak,monthly_min,total_monthly_energy_mwh
0,2025-01,NSW1,7230.060644,12128.62,4188.12,5.378601e+06
1,2025-01,QLD1,6716.615186,11159.32,4351.41,4.996658e+06
2,2025-01,SA1,1252.457724,2485.14,10.55,9.317567e+05
3,2025-01,TAS1,968.286454,1213.70,692.39,7.203218e+05
4,2025-01,VIC1,4408.118995,8733.41,1400.13,3.279366e+06
5,2025-02,NSW1,7493.239926,12210.48,2532.25,5.034871e+06
6,2025-02,QLD1,6656.750834,9384.15,4288.30,4.472773e+06
7,2025-02,SA1,1400.128057,3326.91,-110.47,9.407716e+05
8,2025-02,TAS1,988.957580,1286.10,751.14,6.644999e+05
9,2025-02,VIC1,4829.736203,9490.53,2172.32,3.245191e+06


## Region-level yearly stats from daily_usage

In [5]:
region_daily = (
    df_daily.groupby("region_id").agg(
        avg_daily_demand=("avg_5min_demand", "mean"),
        annual_peak=("daily_peak", "max"),
        annual_min=("daily_min", "min"),
        total_energy_mwh_year=("total_daily_energy_mwh", "sum")
    ).reset_index()
)

region_daily


,region_id,avg_daily_demand,annual_peak,annual_min,total_energy_mwh_year
0,NSW1,6885.281937,9434.59,4534.57,329929.022500
1,QLD1,6192.485824,7808.79,4554.92,296736.050000
2,SA1,917.942512,1598.49,10.55,43989.383333
3,TAS1,1002.235557,1131.23,843.71,48023.988333
4,VIC1,3304.302022,4721.65,1400.13,158332.069167


## Peak day and min day per region

In [6]:
# Peak day (date where daily_peak is maximum)
peak_days = (
    df_daily.loc[df_daily.groupby("region_id")["daily_peak"].idxmax()][
        ["region_id", "date", "daily_peak"]
    ]
    .rename(columns={"date": "peak_day"})
)

# Min day (date where daily_min is minimum)
min_days = (
    df_daily.loc[df_daily.groupby("region_id")["daily_min"].idxmin()][
        ["region_id", "date", "daily_min"]
    ]
    .rename(columns={"date": "min_day"})
)

peak_days, min_days


(  region_id    peak_day  daily_peak
 0      NSW1  2025-01-01     9434.59
 1      QLD1  2025-01-01     7808.79
 7       SA1  2025-01-02     1598.49
 8      TAS1  2025-01-02     1131.23
 9      VIC1  2025-01-02     4721.65,
   region_id     min_day  daily_min
 0      NSW1  2025-01-01    4534.57
 1      QLD1  2025-01-01    4554.92
 2       SA1  2025-01-01      10.55
 8      TAS1  2025-01-02     843.71
 4      VIC1  2025-01-01    1400.13)

## Region-level monthly summary from monthly_usage

In [7]:
region_monthly = (
    df_monthly.groupby("region_id").agg(
        avg_monthly_energy_mwh=("total_monthly_energy_mwh", "mean")
    ).reset_index()
)

region_monthly


,region_id,avg_monthly_energy_mwh
0,NSW1,5.206736e+06
1,QLD1,4.734716e+06
2,SA1,9.362641e+05
3,TAS1,6.924109e+05
4,VIC1,3.262279e+06


## Merge all into region_summary

In [8]:
region_summary = (
    region_daily
    .merge(region_monthly, on="region_id")
    .merge(peak_days[["region_id", "peak_day"]], on="region_id")
    .merge(min_days[["region_id", "min_day"]], on="region_id")
)

region_summary


,region_id,avg_daily_demand,annual_peak,annual_min,total_energy_mwh_year,avg_monthly_energy_mwh,peak_day,min_day
0,NSW1,6885.281937,9434.59,4534.57,329929.022500,5.206736e+06,2025-01-01,2025-01-01
1,QLD1,6192.485824,7808.79,4554.92,296736.050000,4.734716e+06,2025-01-01,2025-01-01
2,SA1,917.942512,1598.49,10.55,43989.383333,9.362641e+05,2025-01-02,2025-01-01
3,TAS1,1002.235557,1131.23,843.71,48023.988333,6.924109e+05,2025-01-02,2025-01-02
4,VIC1,3304.302022,4721.65,1400.13,158332.069167,3.262279e+06,2025-01-02,2025-01-01


## Save region_summary to Postgres

In [9]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS region_summary;"))
    print("🧹 Dropped old region_summary (if existed).")

region_summary.to_sql(
    "region_summary",
    engine,
    index=False,
    if_exists="replace"
)

print("✅ region_summary created with", len(region_summary), "rows.")


🧹 Dropped old region_summary (if existed).
✅ region_summary created with 5 rows.
